# RNN-BOF Demonstration & Usage

## Demo notebook for: Globally Trained Recurrent Neural Network for Forecastingthe Presence of Inpatient Aggression


## Enviroment Details:

### Docker info
- nvcr.io/nvidia/tensorflow:21.03-tf2-py3
- All experiments were performed in the oficial nvidia tensorflow docker container

#### Specific versions of imported libraries can be seen in requirements.txt

## Imports

In [1]:
# processing imports
from processingUtilities import dataFormatting
from processingUtilities import preProcessing
from processingUtilities import windowingFunctions

# modelling and experimental imports
from sklearn.utils import class_weight
from sklearn.metrics import make_scorer
from evaluationUtilities import evalFunctions
from evaluationUtilities import saveLoadResults
from numpy.random import seed
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from RNNBOF import RNNBOFKeras
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
import time
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# loading imports
import pandas as pd

## Seeds

In [2]:
# fix seeds for tensor flow and numpy
seed(1)
tf.random.set_seed(1)
rstate = np.random.RandomState(1) 

## Data Loading

In [5]:
# Load data in, formatted as pandas dataframe containing all entities, an id coulumn, and an ordering column
# The label series/column must be numerical type with 1 representing positive
# All other columns must be numerical (1 hot encoded for catagorical, non ordinal variables)

# eg. fullDataFrame = pd.read_csv('________.csv')

In [ ]:
# manyTimeToDict: Converts a global dataframe containing time stamps and ids into dict of dataframes per entity where keys are the entity IDs.
    # Inputs:
        # sortColumnName: the column to sort on (ascending) 
        # dropID: sepcifies if the id column should be dropped from individual dataframes
        # dataFrame: input global dataframe (contains multiple entities entries over time)
        # idColumn: the column entitiy dataframes are grouped on.
    # Returns:
        # dict of dataframes, with keys representing the entities IDs, ordered by the sorting column

entityDict = dataFormatting.manyTimeToDict(idColumn = 'id', dataFrame = fullDataFrame, sortColumnName = 'sortColumnName', dropID = True)

## Pre-Processing

### Compressed days and missing value treatment

In [6]:
# unCompressor: where our assessments span more than 24 hours, multiple treatment options are possible
    # Inputs:
        # skipTreatment: Type of treatment applied to skipped days.
            # skipTreament = "None": no treatment applied, returns frame as is
            # skipTreament = "Dupe": skipped days are duped for the number of days skipped
            # skipTreament = "None": skipped days are duped for the number of days skipped, but dynamic duped values are replaced with Nan to be interpolated
        # dictOfTDFs: dictionary of dataframes for each entitiy
    # Returns:
        # dict of dataframes with treated skip days in 1 of three ways (skipTreament)
        
# NB: this preProcessing is only applicable in specific circumstances like the ones detailed in the paper.

entityDict2 = preProcessing.unCompressor(dictOfTDFs = entityDict, skipTreatment='None')

In [7]:
# interpolator: interpolates dict of dataframes missing values in one of three ways
    # Inputs:
        # interpolationType: Type if interpolation applied to missing values
            # interpolateType = "None": None interpolation applied
            # interpolateType = "Linear": Linear interpolation applied
            # interpolateType = "Pad": Padding interpolation applied
        # dictOfTDFs: dictionary of dataframes for each entitiy
    # Returns:
        # dict of dataframes with interpolation performed as specifified.

entityDict3 = preProcessing.interpolator(dictOfTDFs = entityDict2, interpolateType='linear')

### Column type identification

In [8]:
#colTyper: takes dict of dataframes returns list of dynamic and catagorical columns and prints stats
    # Inputs:
        # dictOfTDFs: dictionary of dataframes for each entity
    # Returns:
        # names of collumns that change over time, and stay static, accross all entities, as two lists
        
dynamicCols, cataCols = preProcessing.colTyper(dictOfTDFs = entityDict3)

dynamic covariables: 73
catagorical covariables: 40


### Removing entities with short history

In [9]:
# shortIDRemover: Removes all entities with less than a certain amount of entries
    # Inputs:
        # dictOfTDFs: dictionary of dataframes for each entity
        # shortStayLength: minimum length required per entity
    # Returns:
        # dict of dataframes with those shorter than shortStayLength removed
    
# NB: must be at shortest, 3 time stamps longer than than the longest input window length
    
entityDict4 = preProcessing.shortIDRemover(dictOfTDFs = entityDict3, shortStayLength = 30)

removed 23 entities for being less than 30 periods long
total remaining entities: 83


### Data standardization

In [10]:
# standardizeOnTrain: standardize from the windows in the training set, apply to full dataframe.
    # Inputs:
        # dictOfTDFs: dictionary of dataframes for each entity
        # testPercent: percent of ending windows heldout for each entity
        # labelColumn: name of column containing the label binary values
        # holdOutLength: Number of windows removed completly
    # Returns:
        # standardized entity dict
        # dict of columns standard deviation across training data
        # dict of columns mean across training data

# NB: holdOutLength and testPercent must match the ones used in all following functions
    
standEntityDict, stdDict, meanDict = preProcessing.standardizeOnTrain(dictOfTDFs = entityDict4, testPercent = .2, labelColumn = 'anyAggression', holdOutLenght = 1)    

## Windowing and Model Preparation

### Train test split

In [11]:
# timeSeriesPercentTrainTestSplit: splits the full dict of dataframes into two containing the relevent data to be windowed by train and test windower.
    # Inputs:
        # dictOfTDFs: dictionary of dataframes for each entity
        # testPercent: percent of ending windows heldout for each entity
        # holdOutLength: Number of windows removed completly
        # MaxInputWindow: input window length
    # Returns:
        # Dict of dataframes per entity for train and test sets

# NB: this is the split process documented in the accompanying paper (Section 3.3) and the test dict contains the windowed days into the train set - the holdout length
    
trainDict10, testDict10 = preProcessing.timeSeriesPercentTrainTestSplit(dictOfTDFs = standEntityDict, testPercent = .2, MaxInputWindow = 10 , holdOutLength = 1)

### Train and test folds (Dataframe version, used by sklearn and LightGBM models and FFNN in keras)

In [12]:
# trainWindowerRolling: returns the rolling training windows as described in section 3.3 and crossvalidation fold data, using coulumn names to represent sequential data
    # Inputs:
        # trainingDfDict: training data from timeSeriesPercentTrainTestSplit as a dict of dataframes
        # numberOfFolds: number of folds for sequential cross validation (section 3.3 and 4.4). will create numberOfFolds + 1 pools for numberOfFolds sequential evaluations
        # windowLength: input window length
        # dynamicCols: dynamic column names list as created by colTyper
        # cataCols: static column names list as created by colTyper
        # predictionVariable: name of column containing the label binary values
    # Returns:
        # dataframe of training windows, with columns named: name_lag_0, name_lag_1 ....  with the number representing how many iterations back in time that variable represents
        # np.array of the labels corresponding to each window in the dataframe
        # array of arrays to be used by the custom cross validation function, containg the DF index for each cross validations folds
        
# NB: this function needs to be optimised, it currently is extermely slow.
# NB: This is the datafram/flat version as used by all non iterative models such as ours.

x_train10, y_train10, foldIdx10 = windowingFunctions.trainWindowerRolling(trainingDfDict = trainDict10, numberOfFolds = 5, windowLength = 10, dynamicCols = dynamicCols, cataCols = cataCols, predictionVariable = 'anyAggression')

In [13]:
# testWindower: returns the rolling trainign windows as described in section 3.3
    # Inputs:
        # testingDfDict: testing data from timeSeriesPercentTrainTestSplit as a dict of dataframes
        # windowLength: input window length
        # dynamicCols: dynamic column names list as created by colTyper
        # cataCols: static column names list as created by colTyper
        # predictionVariable: name of column containing the label binary values
    # Returns:
        # dataframe of testing windows, with columns named: name_lag_0, name_lag_1 ....  with the number representing how many iterations back in time that variable represents
        # np.array of the labels corresponding to each window in the dataframe
        
# NB: this function needs to be optimised, it currently is extermely slow.

x_test10, y_test10 = windowingFunctions.testWindower(testingDfDict = testDict10, windowLength = 10, dynamicCols = dynamicCols, cataCols = cataCols, predictionVariable = 'anyAggression')

In [14]:
print('DataFrame 10 shape:')
print('x train shape:', x_train10.shape)
print('x test shape:', x_test10.shape)

DataFrame 10 shape:
x train shape: (8513, 780)
x test shape: (2213, 780)


### Train and test folds (3D Arrays version, used by RNN-BOF)

In [15]:
# arrayTrainWindowerRolling: returns the rolling trainign windows as described in section 3.3 and crossvalidation fold data, using arrays to represent sequential windows
    # Inputs:
        # trainingDfDict: training data from timeSeriesPercentTrainTestSplit as a dict of dataframes
        # numberOfFolds: number of folds for sequential cross validation (section 3.3 and 4.4). will create numberOfFolds + 1 pools for numberOfFolds sequential evaluations
        # windowLength: input window length
        # predictionVariable: name of column containing the label binary values
        # timeFirst: indication for if window arrays should be time or feature first
            # timeFirst = True: output.shape = (window_number, timestep, feature)
            # timeFirst = False: output.shape = (window_number, feature, timestep)
            # True is used by RNN-BOF
        # sliding: indication for if windows should be sliding or non-overlapping
            # True is used by RNN-BOF
        # Univariate: Use True if data is univariate
        # removeNaNY: If true, removes windows and labels if the label is missing
            # redundant due to other pre-processing steps
    # Returns:
        # 3D np.array representing windows, timesteps and features
        # np.array of the labels corresponding to each window in the dataframes
        # array of arrays to be used by the custom cross validation function, containg the DF index for each cross validations folds

ax_train10, ay_train10, afoldIdx10 = windowingFunctions.arrayTrainWindowerRolling(trainingDfDict = trainDict10, numberOfFolds = 5, windowLength =  10, predictionVariable = 'anyAggression', 
                                                                            timeFirst = True, sliding = True, univariate = False, removeNaNY = True )

In [16]:
# arrayTestWindower: returns the rolling testing windows as described in section 3.3 and crossvalidation fold data, using arrays to represent sequential windows
    # Inputs:
        # testingDfDict: testing data from timeSeriesPercentTrainTestSplit as a dict of dataframes
        # windowLength: input window length
        # predictionVariable: name of column containing the label binary values
        # timeFirst: indication for if window arrays should be time or feature first
            # timeFirst = True: output.shape = (window_number, timestep, feature)
            # timeFirst = False: output.shape = (window_number, feature, timestep)
            # True is used by RNN-BOF
        # sliding: indication for if windows should be sliding or non-overlapping
            # True is used by RNN-BOF
        # Univariate: Use True if data is univariate
        # removeNaNY: If true, removes windows and labels if the label is missing
            # redundant due to other pre-processing steps
    # Returns:
        # 3D np.array representing windows, timesteps and features
        # np.array of the labels corresponding to each window in the dataframe

ax_test10, ay_test10 = windowingFunctions.arrayTestWindower(testingDfDict = testDict10, windowLength = 10, predictionVariable =  'anyAggression', 
                                                      timeFirst = True, sliding = True, univariate =  False, removeNaNY =  True )


In [17]:
print('Array 10 shape:')
print('train shape:', ax_train10.shape)
print('test shape:', ax_test10.shape)

Array 10 shape:
train shape: (8513, 10, 114)
test shape: (2213, 10, 114)


## RNN-BOF experiments

### Global params

In [18]:
# Comput class wieghts on the training set

class_weights = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.asarray([0,1]), y = ay_train10)
class_weightDict = {0: class_weights[0],
                1: class_weights[1]}

In [19]:
# define scoring function
# we use area under precision recall curve

auprg_scorer = make_scorer(evalFunctions.auprg, greater_is_better = True)

In [20]:
# defining set variables used across

# input shape of a single window for a single entity.
inputShape = ax_train10.shape[1:3]
print('inputShape = ' + str(inputShape))

# batch size used for training model
batch_size = 512

inputShape = (10, 114)


### Hyperopt Tuning

In [21]:
%%capture
# use line magic to suprress outputs to stop jupyter notebook crashing
# can be removed for small amounts of iterations/folds/epochs




tic = time.time()


def objective(params):
    
    # define lambda build function, from the rnnbof keras model
    # takes params identified in the param space
    # ranges can be defined below
    build_func = lambda: RNNBOFKeras.get_lstm(shape = inputShape, 
                                              numLayers = params['number_of_layers'], 
                                              dropout = params['dropout'], 
                                              l2Val = params['l2'], 
                                              learnRate = 0.0001, 
                                              numHiddenNodes = params['hidden_nodes'])
    
    # build specific params
    buildParams = {
        'epochs': int(params['epochs']),
        'batch_size': batch_size}
    
    # build sklearn wrapper for hyperopt tuning
    rnnBofModel = KerasRegressor(build_fn = build_func, 
                                 verbose = -1, 
                                 **buildParams)
    
    # calculate score from built model using cros_val_score
    # uses the custom cross validation function definied in windowingFunctions
    # uses the auprg_scorer using https://github.com/meeliskull/prg implimentation
    score = -(cross_val_score(rnnBofModel, 
                              ax_train10, 
                              ay_train10, 
                              scoring = auprg_scorer, 
                              fit_params = {'class_weight':class_weightDict}, 
                              cv = windowingFunctions.custom_idx_folder_rolling(afoldIdx10),
                              verbose = 0).mean()
             )
    
    return(score)
    
toc = time.time()

tune_time_taken = tic-toc

In [22]:
# defined param space used in experiments
paramSpace = {   
     'epochs': hp.quniform('epochs', 10, 100, 1),
     'hidden_nodes': hp.quniform('hidden_nodes', 10, 100, 1),
     'number_of_layers' : hp.quniform('number_of_layers', 1,4,1),
     'dropout': hp.quniform('dropout',0.1,0.5, 0.01),
     'l2': hp.quniform('l2', 0.000001, 0.001, 0.000001)
}

In [ ]:
# returns best hyperParams
best = fmin(
    fn = objective,
    space = paramSpace,
    algo = tpe.suggest,
    max_evals = 100,
    rstate = rstate
)    
# NB: returns some variables as wrong type: eg int for number_of_layers as a float.

In [23]:
# best identified during the 100 iterations of hyperopt ran for the experiment in the paper
best = {'dropout': 0.5,
 'epochs': 87.0,
 'hidden_nodes': 21.0,
 'l2': 0.000878,
 'number_of_layers': 1.0}

### Final train

In [24]:
# Define the final RNNBOF keras model based on the best identified parameters
finalRNNBOF10 = RNNBOFKeras.get_lstm(shape = inputShape, 
                                      numLayers = best['number_of_layers'], 
                                      dropout = best['dropout'], 
                                      l2Val = best['l2'], 
                                      learnRate = 0.0001, 
                                      numHiddenNodes = best['hidden_nodes'])

In [25]:
tic = time.time()

# Final RNNBOF train, based on the best identified parameters
history = finalRNNBOF10.fit(
        ax_train10,
        ay_train10,
        epochs=int(best['epochs']),
        batch_size = 512,
        class_weight=class_weightDict
)

toc = time.time()
final_train_time_taken = toc - tic


Epoch 1/87
17/17 [==============================] - 2s 3ms/step - loss: 0.8537
Epoch 2/87
17/17 [==============================] - 0s 3ms/step - loss: 0.8387
Epoch 3/87
17/17 [==============================] - 0s 3ms/step - loss: 0.8109
Epoch 4/87
17/17 [==============================] - 0s 3ms/step - loss: 0.7663
Epoch 5/87
17/17 [==============================] - 0s 3ms/step - loss: 0.7600
Epoch 6/87
17/17 [==============================] - 0s 3ms/step - loss: 0.7273
Epoch 7/87
17/17 [==============================] - 0s 3ms/step - loss: 0.7201
Epoch 8/87
17/17 [==============================] - 0s 3ms/step - loss: 0.7114
Epoch 9/87
17/17 [==============================] - 0s 3ms/step - loss: 0.6768
Epoch 10/87
17/17 [==============================] - 0s 3ms/step - loss: 0.6993
Epoch 11/87
17/17 [==============================] - 0s 3ms/step - loss: 0.6848
Epoch 12/87
17/17 [==============================] - 0s 3ms/step - loss: 0.6484
Epoch 13/87
17/17 [==============================

### Make Predictions

In [26]:
# make predictions on the held out test set

probabilitiesPred = finalRNNBOF10.predict(ax_test10)
probabilitiesPredArray  = np.asarray([float(x) for x in probabilitiesPred])

### Save Results

In [34]:
import pickle

# simple pickle results saver as a dictionary
def resultsSave(modelName, bParams, trueValues, probaPred, history, tune_time, ftrain_time):
    
    saveName = modelName
    
    modelName = dict()

    modelName['model_params'] = bParams

    modelName['model_proba'] = (trueValues, probaPred)
    
    modelName['tf_history'] = history
    
    modelName['tune_train_time'] = (tune_time, ftrain_time)

    filename = 'Results/'+saveName
    
    with open(filename, 'wb')as fp:
        
        pickle.dump(modelName, fp)

In [40]:
saveLoadResults.resultsSave(modelName = 'RNNBOF10', 
                          bParams = best, 
                          trueValues = ay_test10, 
                          probaPred = probabilitiesPredArray, 
                          history = history.params, 
                          tune_time = tune_time_taken, 
                          ftrain_time = final_train_time_taken)